In [1]:
# Cell 1: Setup & Environment Installation

# --- ติดตั้ง Libraries ---
# ultralytics: Library หลักสำหรับ YOLOv8
# opendatasets: สำหรับดาวน์โหลดข้อมูลจาก Kaggle
# pycocotools: เครื่องมือสำหรับคำนวณ mAP (YOLO มักจะติดตั้งให้โดยอัตโนมัติ)
!pip install ultralytics opendatasets --quiet

# --- Import Libraries ที่จำเป็น ---
import os
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import shutil
import yaml
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt

print("Setup Complete. YOLOv8 is ready.")


DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Setup Complete. YOLOv8 is ready.


In [2]:
# Cell 2: Data Acquisition
import opendatasets as od

# URL ของ Dataset บน Kaggle
dataset_url = 'https://www.kaggle.com/c/global-wheat-detection'

# ดาวน์โหลดข้อมูล (จะมีการถามหา username และ key จาก kaggle.json)
od.download(dataset_url)

# --- กำหนด Path หลัก ---
DATA_DIR = './global-wheat-detection/'
TRAIN_IMG_DIR = os.path.join(DATA_DIR, 'train')
TEST_IMG_DIR = os.path.join(DATA_DIR, 'test')
TRAIN_CSV_PATH = os.path.join(DATA_DIR, 'train.csv')

print(f"Data downloaded to: {DATA_DIR}")


Skipping, found downloaded files in ".\global-wheat-detection" (use force=True to force download)
Data downloaded to: ./global-wheat-detection/


In [3]:
# Cell 3: Explore and Process Annotations
# อ่านไฟล์ CSV
df = pd.read_csv(TRAIN_CSV_PATH)

# แปลงคอลัมน์ 'bbox' จาก string เป็น list ของตัวเลข
df['bbox'] = df['bbox'].apply(lambda x: eval(x))

# แยก bbox ออกเป็นแต่ละคอลัมน์
df[['x_min', 'y_min', 'width', 'height']] = pd.DataFrame(df['bbox'].tolist(), index=df.index)

# ฟังก์ชันสำหรับแปลง Bbox เป็น YOLO format
def to_yolo_format(box, img_w, img_h):
    x_min, y_min, w, h = box
    x_center = (x_min + w / 2) / img_w
    y_center = (y_min + h / 2) / img_h
    w_norm = w / img_w
    h_norm = h / img_h
    return f"0 {x_center} {y_center} {w_norm} {h_norm}" # class_id คือ 0

# สร้างโฟลเดอร์สำหรับเก็บข้อมูลที่แปลงแล้ว
os.makedirs('dataset/labels/train', exist_ok=True)
os.makedirs('dataset/images/train', exist_ok=True)

# วนลูปเพื่อสร้างไฟล์ .txt สำหรับแต่ละภาพ
for image_id, group in tqdm(df.groupby('image_id'), desc="Converting to YOLO format"):
    # สร้างไฟล์ label
    label_path = f"dataset/labels/train/{image_id}.txt"
    with open(label_path, 'w') as f:
        for _, row in group.iterrows():
            img_w, img_h = row['width'], row['height']
            bbox = [row['x_min'], row['y_min'], row['width'], row['height']]
            yolo_str = to_yolo_format(bbox, img_w, img_h)
            f.write(yolo_str + '\n')

    # คัดลอกไฟล์ภาพ
    shutil.copy(f"{TRAIN_IMG_DIR}/{image_id}.jpg", f"dataset/images/train/{image_id}.jpg")

print("\nConversion to YOLO format complete.")


Converting to YOLO format:   0%|          | 0/3373 [00:00<?, ?it/s]


Conversion to YOLO format complete.


In [4]:
# Cell 4: Create Train/Val Split and data.yaml

# สร้างโฟลเดอร์สำหรับ validation set
os.makedirs('dataset/images/val', exist_ok=True)
os.makedirs('dataset/labels/val', exist_ok=True)

# รายชื่อไฟล์ทั้งหมด
all_images = sorted(os.listdir('dataset/images/train'))
np.random.shuffle(all_images) # สุ่มลำดับไฟล์

# แบ่ง 20% เป็น validation set
val_size = int(len(all_images) * 0.2)
val_images = all_images[:val_size]
train_images = all_images[val_size:]

# ย้ายไฟล์ validation
for img_name in tqdm(val_images, desc="Creating validation set"):
    base_name = os.path.splitext(img_name)[0]
    # ย้ายไฟล์ภาพ
    shutil.move(f'dataset/images/train/{img_name}', f'dataset/images/val/{img_name}')
    # ย้ายไฟล์ label
    shutil.move(f'dataset/labels/train/{base_name}.txt', f'dataset/labels/val/{base_name}.txt')

print(f"\nCreated train/val split. Train size: {len(train_images)}, Val size: {len(val_images)}")

# --- สร้างไฟล์ data.yaml ---
data_yaml = {
    'train': '../dataset/images/train',
    'val': '../dataset/images/val',
    'nc': 1, # Number of classes
    'names': ['wheat'] # Class names
}

with open('data.yaml', 'w') as f:
    yaml.dump(data_yaml, f, default_flow_style=False)

print("\ndata.yaml file created successfully.")


Creating validation set:   0%|          | 0/674 [00:00<?, ?it/s]


Created train/val split. Train size: 2699, Val size: 674

data.yaml file created successfully.


In [5]:
# Cell 5: Fix Label Format and Train the YOLOv8 Model

# First, let's fix the label format issue
# Remove the existing cache to force re-processing
import glob
cache_files = glob.glob('dataset/**/*.cache', recursive=True)
for cache_file in cache_files:
    if os.path.exists(cache_file):
        os.remove(cache_file)
        print(f"Removed cache file: {cache_file}")

# Re-create labels with correct image dimensions
print("Fixing label format...")
for image_id, group in tqdm(df.groupby('image_id'), desc="Re-creating YOLO labels"):
    # Get actual image dimensions
    img_path = f"{TRAIN_IMG_DIR}/{image_id}.jpg"
    if not os.path.exists(img_path):
        continue
        
    img = cv2.imread(img_path)
    if img is None:
        continue
        
    actual_img_h, actual_img_w = img.shape[:2]
    
    # Create label file
    if image_id in [img.split('.')[0] for img in train_images]:
        label_path = f"dataset/labels/train/{image_id}.txt"
    else:
        label_path = f"dataset/labels/val/{image_id}.txt"
    
    with open(label_path, 'w') as f:
        for _, row in group.iterrows():
            bbox = [row['x_min'], row['y_min'], row['width'], row['height']]
            yolo_str = to_yolo_format(bbox, actual_img_w, actual_img_h)
            f.write(yolo_str + '\n')

print("Label format fixed.")

Removed cache file: dataset\labels\train.cache
Removed cache file: dataset\labels\val.cache
Fixing label format...


Re-creating YOLO labels:   0%|          | 0/3373 [00:00<?, ?it/s]

Label format fixed.


In [6]:
# โหลดโมเดล YOLOv8s ที่ pre-trained แล้ว
model = YOLO('yolov8s.pt')

# เริ่มการฝึกสอนโมเดล
# epochs=25 เป็นค่าเริ่มต้นที่ดีสำหรับการทดลอง (โจทย์จริงอาจใช้ 50-100)
# imgsz=640 คือขนาดภาพที่จะใช้เทรน
results = model.train(
    data='data.yaml',
    epochs=25,
    imgsz=640,
    project='runs',
    name='wheat_detection_exp1'
)

print("\nTraining complete.")
print("Model weights and results are saved in 'runs/detect/wheat_detection_exp1'")


Ultralytics 8.3.161  Python-3.12.0 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=25, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=wheat_detection_exp14, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, p

train: Scanning C:\Users\potij\OneDrive\เดสก์ท็อป\supA\เก็ง\CV\Object Detection\dataset\labels\train... 2699 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2699/2699 [00:07<00:00, 381.07it/s]


train: New cache created: C:\Users\potij\OneDrive\\supA\\CV\Object Detection\dataset\labels\train.cache
val: Fast image access  (ping: 0.20.1 ms, read: 148.2152.1 MB/s, size: 183.7 KB)


val: Scanning C:\Users\potij\OneDrive\เดสก์ท็อป\supA\เก็ง\CV\Object Detection\dataset\labels\val... 1208 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1208/1208 [00:02<00:00, 426.25it/s]


val: New cache created: C:\Users\potij\OneDrive\\supA\\CV\Object Detection\dataset\labels\val.cache
Plotting labels to runs\wheat_detection_exp14\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\wheat_detection_exp14
Starting training for 25 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/169 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 26.00 MiB. GPU 0 has a total capacity of 6.00 GiB of which 3.16 GiB is free. Of the allocated memory 1.73 GiB is allocated by PyTorch, and 42.83 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# Cell 6: Inference on a Sample Image

# Path ไปยังโมเดลที่ดีที่สุดที่เทรนเสร็จแล้ว
BEST_MODEL_PATH = 'runs/detect/wheat_detection_exp1/weights/best.pt'

# โหลดโมเดลที่เราเทรนเอง
model = YOLO(BEST_MODEL_PATH)

# เลือกภาพตัวอย่างจาก validation set
sample_image_path = os.path.join('dataset/images/val', np.random.choice(os.listdir('dataset/images/val')))

# ทำนายผล
results = model(sample_image_path)

# วาด Bounding Box ลงบนภาพ
annotated_img = results[0].plot()

# แสดงผล
plt.figure(figsize=(12, 12))
plt.imshow(cv2.cvtColor(annotated_img, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title(f"Inference result for {os.path.basename(sample_image_path)}")
plt.show()


In [ ]:
# Cell 7: Generate Submission File

# โหลดโมเดลที่ดีที่สุด
model = YOLO(BEST_MODEL_PATH)

# รายชื่อภาพใน test set
test_images = os.listdir(TEST_IMG_DIR)

submission_data = []

# วนลูปทำนายผลใน test set
for img_name in tqdm(test_images, desc="Generating predictions for submission"):
    img_path = os.path.join(TEST_IMG_DIR, img_name)
    image_id = os.path.splitext(img_name)[0]
    
    # ทำนายผล
    results = model(img_path, verbose=False) # verbose=False เพื่อไม่ให้แสดง log เยอะ
    
    prediction_string = []
    # ดึงค่า bbox (xyxy) และ confidence
    boxes = results[0].boxes.xyxy.cpu().numpy() # [x_min, y_min, x_max, y_max]
    confs = results[0].boxes.conf.cpu().numpy()
    
    for box, conf in zip(boxes, confs):
        x_min, y_min, x_max, y_max = box
        width = x_max - x_min
        height = y_max - y_min
        # จัดรูปแบบ string: "conf x_min y_min w h"
        prediction_string.append(f"{conf:.4f} {x_min:.2f} {y_min:.2f} {width:.2f} {height:.2f}")
        
    # รวมทุก bbox ในภาพเดียวกัน
    prediction_string = " ".join(prediction_string)
    submission_data.append({'image_id': image_id, 'PredictionString': prediction_string})

# สร้าง DataFrame และบันทึกเป็น CSV
submission_df = pd.DataFrame(submission_data)
submission_df.to_csv('submission.csv', index=False)

print("\nsubmission.csv created successfully!")
print("Top 5 rows of the submission file:")
display(submission_df.head())
